In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
from Finance.Finance import StockScreener
import numpy as np

screener = StockScreener()

df = screener.df_NASDAQ

print(df.columns)
print(df.head())
print('Total:', len(df))
df = df[df['LastSale']!='n/a']
df['LastSale'] = df['LastSale'].astype(float).fillna(0.0)
print('0.9-3.0: ', len(df[np.logical_and(df['LastSale']<3.0, df['LastSale']>0.9)]))
print('0.1-0.5: ', len(df[np.logical_and(df['LastSale']<0.5, df['LastSale']>0.1)]))


premarket_actives = screener.FindPremarketMostActives(price_min=0.3, price_max=2.5, n_trans_min=5)
#aftermarket_actives = screener.FindAftermarketMostActives(price_min=0.9, price_max=2.5, n_trans_min=10)


In [ ]:
from Finance.Finance import StockScreener

stocksWatchlist = ['NASDAQ:ORIG', 'NASDAQ:BCRX', 'NYSEMKT:TXMD']

screener = StockScreener(watchlist=stocksWatchlist)

df = screener.getIntradayOrHistoricPrices(r'NASDAQ:AMD', '60', '25m')
#print(df)
df = screener.getIntradayOrHistoricPrices(r'NYSE:SGY', '1800', '30d')
#print(df)
df = screener.getIntradayOrHistoricPrices(r'NYSEMKT:RVP', '1800', '30d')
#print(df)

try:
    screener.StartGoogleFinanceWatchlist()
    screener.updatePrices()
except:
    pass
finally:
    try:
        #screener.CloseDriver()
        pass
    except:
        pass


In [ ]:
from Finance.Finance import StockScreener

screener = StockScreener()

#screener.SetupYahooGainerLoserWatch() 
gl_symbs = screener.GetYahooGainersLosers() 

print(gl_symbs)

In [ ]:
screener.GetNotableGainerLoserStocks(gl_symbs[0], price_range=[1.0, 10.0]
            ,volPrice_avg=50000, close_open_jump_perc_max=10)

In [ ]:
from multiprocessing import Process, Pipe, Queue
from Finance.Finance import StockScreener
import time

conn0, conn1 = Pipe()

yg_screener = StockScreener(commPipe=conn1, vbose=False)

yg_screener.start()

intraday_screener = StockScreener(vbose=False)
intraday_screener.SetStocksToTrack('YGainers', [])

time_last_YGL = time.perf_counter()
refresh_YGL_every_n_sec = 2*60
time_last_clean = time.perf_counter()
clean_trackedList_every_n_sec = 10*60

n_recv = 0
conn0.send(['monitorYGL'])
n_send = 1

Gainers = []
Losers = []

t_sleep = 5
got_gl_list = False

modelFunc_psr = intraday_screener.ModelPriceOnly_SharpRise
params_psr = {'df':intraday_screener.paramTypes.intradaylive}

modelFunc_sr = intraday_screener.ModelSharpRise
params_sr = {'df':intraday_screener.paramTypes.intradaylive}

modelFunc_sf = intraday_screener.ModelintVdPSharpFall
params_sf = {'df':intraday_screener.paramTypes.intradaylive}

modelFunc_si = intraday_screener.ModelintVdPStopThenIncrease
params_si = {'df':intraday_screener.paramTypes.intradaylive}

while True:
    intraday_screener.UpdateTrackedStockData('YGainers')
    #intraday_screener.GetIndicatorsForTrackedStocks('YGainers', \
    #                    pv_th_1=30.0e3, pv_th_60=5.0e5, dp_th=0.01, d_pv_th=1.0)
    intraday_screener.ApplyRealtimeModelToTrackedStocks('YGainers', modelFunc_psr, params_psr, modelName='PRICEONLY-SHARP-RISE model', alarm='DoorBuzzer')
    intraday_screener.ApplyRealtimeModelToTrackedStocks('YGainers', modelFunc_sr, params_sr, modelName='SHARP-RISE model', alarm='DoorBuzzer')
    intraday_screener.ApplyRealtimeModelToTrackedStocks('YGainers', modelFunc_sf, params_sf, modelName='SHARP-FALL model', alarm='DoorBuzzer')
    intraday_screener.ApplyRealtimeModelToTrackedStocks('YGainers', modelFunc_si, params_si, modelName='STOPINCREASE model', alarm='StoreDoor')

    tima_elapsed_clean = time.perf_counter() - time_last_clean
    if tima_elapsed_clean>clean_trackedList_every_n_sec:
        intraday_screener.cleanTrakedStocks('YGainers')
        time_last_clean = time.perf_counter()

    tima_elapsed = time.perf_counter() - time_last_YGL
    if got_gl_list and tima_elapsed>refresh_YGL_every_n_sec:
        conn0.send(['monitorYGL'])
        time_last_YGL = time.perf_counter()
        n_send += 1
    if conn0.poll():
        resp = conn0.recv()
        n_recv += 1
        assert resp[0]=='data' and resp[1]=='GLSYMBOLS'
        gs_ = resp[2][0]
        intraday_screener.AddStockToTrackList('YGainers', gs_)
        if not got_gl_list:
            got_gl_list = True
    time.sleep(t_sleep)
    if got_gl_list:
        t_sleep = 30

intraday_screener.SoundAlarm(n=1, p=0)

conn0.send(['quit'])

screener.join()


In [ ]:
from Finance.Finance import *
import datetime
import numpy as np

screener = StockScreener()

noticable_dic = screener.FindNoticableIntradayHistoricalSymbols(period_days=50, perc_change=10,  mins_change=10, GL='G')
print(len(noticable_dic), noticable_dic)

In [ ]:
%pylab inline

from Finance.Finance import *
import datetime
import numpy as np
import pandas as pd

screener = StockScreener()

# 20%
#noticable_dic = {'NASDAQ:ZSAN': [datetime.date(2017, 2, 13), datetime.date(2017, 2, 24)], 'NASDAQ:JAGX': [datetime.date(2017, 2, 27)], 'NASDAQ:TBIO': [datetime.date(2017, 2, 9)], 'NYSEMKT:ESNC': [datetime.date(2017, 3, 2)], 'NASDAQ:VNRCP': [datetime.date(2017, 2, 2)], 'NASDAQ:PHMD': [datetime.date(2017, 2, 14)], 'NASDAQ:PSIX': [datetime.date(2017, 2, 16)], 'NASDAQ:GLBS': [datetime.date(2017, 1, 17), datetime.date(2017, 1, 18)], 'NASDAQ:CBIO': [datetime.date(2017, 2, 13)], 'NASDAQ:QBAK': [datetime.date(2017, 2, 15)], 'NYSEMKT:ENRJ': [datetime.date(2017, 1, 20)], 'NASDAQ:OPGN': [datetime.date(2017, 2, 14)], 'NASDAQ:SKYS': [datetime.date(2017, 2, 9)], 'NYSEMKT:DSS': [datetime.date(2017, 2, 3)], 'NASDAQ:NFEC': [datetime.date(2017, 1, 19), datetime.date(2017, 2, 28)], 'NASDAQ:EVAR': [datetime.date(2017, 1, 20)], 'NASDAQ:VRAY': [datetime.date(2017, 2, 27)], 'NASDAQ:VNRBP': [datetime.date(2017, 2, 2), datetime.date(2017, 2, 3)], 'NASDAQ:FXCM': [datetime.date(2017, 2, 7)], 'NASDAQ:RNVA': [datetime.date(2017, 2, 17), datetime.date(2017, 3, 3)], 'NASDAQ:ETRM': [datetime.date(2017, 1, 20)], 'NASDAQ:ESES': [datetime.date(2017, 1, 27), datetime.date(2017, 1, 30)], 'NASDAQ:VNR': [datetime.date(2017, 2, 2), datetime.date(2017, 2, 7)], 'NASDAQ:KBSF': [datetime.date(2017, 2, 9), datetime.date(2017, 2, 10), datetime.date(2017, 2, 22)], 'NYSE:XNY': [datetime.date(2017, 1, 23)], 'NASDAQ:KTOV': [datetime.date(2017, 2, 9)], 'NASDAQ:DFFN': [datetime.date(2017, 2, 2), datetime.date(2017, 2, 3)], 'NYSEMKT:PIP': [datetime.date(2017, 2, 6)], 'NASDAQ:INAP': [datetime.date(2017, 2, 22)], 'NASDAQ:MEMP': [datetime.date(2017, 1, 17), datetime.date(2017, 2, 16)], 'NASDAQ:PULM': [datetime.date(2017, 1, 17), datetime.date(2017, 1, 26), datetime.date(2017, 2, 1), datetime.date(2017, 2, 2)], 'NASDAQ:MYOS': [datetime.date(2017, 2, 3), datetime.date(2017, 2, 7)], 'NASDAQ:AMTX': [datetime.date(2017, 1, 19)], 'NASDAQ:ZAIS': [datetime.date(2017, 2, 14)], 'NASDAQ:CHCI': [datetime.date(2017, 1, 18)], 'NASDAQ:VNRAP': [datetime.date(2017, 2, 2)], 'NASDAQ:HTGM': [datetime.date(2017, 2, 27)], 'NYSEMKT:MCZ': [datetime.date(2017, 2, 3)], 'NASDAQ:MBRX': [datetime.date(2017, 2, 1)], 'NASDAQ:CERU': [datetime.date(2017, 2, 3)], 'NYSEMKT:BGI': [datetime.date(2017, 1, 24)], 'NYSE:EMG': [datetime.date(2017, 1, 19)], 'NASDAQ:VRML': [datetime.date(2017, 2, 21)], 'NASDAQ:MOSY': [datetime.date(2017, 2, 17)], 'NASDAQ:NVFY': [datetime.date(2017, 2, 21), datetime.date(2017, 2, 23)], 'NASDAQ:BASI': [datetime.date(2017, 1, 19), datetime.date(2017, 2, 14), datetime.date(2017, 2, 27), datetime.date(2017, 2, 28)], 'NASDAQ:XGTI': [datetime.date(2017, 2, 8)], 'NASDAQ:OCRX': [datetime.date(2017, 2, 8)], 'NASDAQ:RLOG': [datetime.date(2017, 2, 14)], 'NASDAQ:GALT': [datetime.date(2017, 2, 22)], 'NASDAQ:FNCX': [datetime.date(2017, 1, 25), datetime.date(2017, 2, 23)], 'NASDAQ:SGNL': [datetime.date(2017, 2, 7)], 'NASDAQ:WCST': [datetime.date(2017, 2, 8)], 'NYSEMKT:IGC': [datetime.date(2017, 2, 22)], 'NASDAQ:RLJE': [datetime.date(2017, 1, 18)], 'NASDAQ:RGSE': [datetime.date(2017, 1, 30)], 'NASDAQ:FCEL': [datetime.date(2017, 1, 26)], 'NASDAQ:NETE': [datetime.date(2017, 1, 19), datetime.date(2017, 2, 8)], 'NASDAQ:SINO': [datetime.date(2017, 2, 13)], 'NASDAQ:LBIX': [datetime.date(2017, 2, 17)], 'NASDAQ:INFI': [datetime.date(2017, 3, 2)], 'NASDAQ:AEHR': [datetime.date(2017, 2, 8)], 'NASDAQ:BNTC': [datetime.date(2017, 1, 17), datetime.date(2017, 2, 2), datetime.date(2017, 2, 7)], 'NASDAQ:APRI': [datetime.date(2017, 1, 18)], 'NYSEMKT:TEUM': [datetime.date(2017, 2, 23)], 'NYSEMKT:APHB': [datetime.date(2017, 2, 8)], 'NASDAQ:MRNS': [datetime.date(2017, 1, 17)], 'NASDAQ:ACUR': [datetime.date(2017, 2, 7), datetime.date(2017, 2, 8), datetime.date(2017, 2, 10)], 'NASDAQ:NOVN': [datetime.date(2017, 1, 27), datetime.date(2017, 2, 15)], 'NYSEMKT:NAVB': [datetime.date(2017, 2, 23)], 'NASDAQ:NAKD': [datetime.date(2017, 2, 2), datetime.date(2017, 2, 6)], 'NASDAQ:MSDI': [datetime.date(2017, 1, 26)], 'NASDAQ:PPHM': [datetime.date(2017, 2, 17)], 'NYSE:SSI': [datetime.date(2017, 3, 2)], 'NASDAQ:CRDS': [datetime.date(2017, 1, 19), datetime.date(2017, 2, 13)], 'NASDAQ:DRYS': [datetime.date(2017, 1, 30)], 'NASDAQ:EYEG': [datetime.date(2017, 2, 21)], 'NASDAQ:SBBP': [datetime.date(2017, 3, 2)], 'NASDAQ:CIDM': [datetime.date(2017, 1, 17), datetime.date(2017, 2, 15)], 'NASDAQ:OPHC': [datetime.date(2017, 1, 17)], 'NASDAQ:APOP': [datetime.date(2017, 2, 2), datetime.date(2017, 2, 14)], 'NASDAQ:DELT': [datetime.date(2017, 1, 19)], 'NASDAQ:CUR': [datetime.date(2017, 2, 16)]}

# 10%
noticable_dic = {'NASDAQ:ZSAN': [datetime.date(2017, 1, 31), datetime.date(2017, 2, 13), datetime.date(2017, 2, 14), datetime.date(2017, 2, 15), datetime.date(2017, 2, 16), datetime.date(2017, 2, 24)], 'NYSEMKT:TGB': [datetime.date(2017, 2, 17)], 'NASDAQ:MNTX': [datetime.date(2017, 1, 17)], 'NYSEMKT:ESNC': [datetime.date(2017, 3, 2)], 'NASDAQ:VNRCP': [datetime.date(2017, 2, 2), datetime.date(2017, 2, 3)], 'NYSE:HGG': [datetime.date(2017, 2, 6), datetime.date(2017, 2, 8), datetime.date(2017, 2, 24)], 'NASDAQ:GLBS': [datetime.date(2017, 1, 18), datetime.date(2017, 1, 19), datetime.date(2017, 1, 23), datetime.date(2017, 1, 24), datetime.date(2017, 1, 26), datetime.date(2017, 2, 13)], 'NASDAQ:QBAK': [datetime.date(2017, 2, 15)], 'NASDAQ:PRKR': [datetime.date(2017, 1, 20)], 'NASDAQ:BIOC': [datetime.date(2017, 1, 19), datetime.date(2017, 2, 8), datetime.date(2017, 2, 10), datetime.date(2017, 2, 22)], 'NYSE:LXU': [datetime.date(2017, 2, 28)], 'NASDAQ:VNRBP': [datetime.date(2017, 2, 2), datetime.date(2017, 2, 3)], 'NASDAQ:DGLY': [datetime.date(2017, 1, 30)], 'NASDAQ:ETRM': [datetime.date(2017, 1, 20), datetime.date(2017, 1, 24), datetime.date(2017, 2, 1), datetime.date(2017, 2, 2)], 'NASDAQ:DXTR': [datetime.date(2017, 1, 17)], 'NASDAQ:INAP': [datetime.date(2017, 2, 22)], 'NASDAQ:OPHC': [datetime.date(2017, 1, 17)], 'NASDAQ:HMNY': [datetime.date(2017, 1, 23), datetime.date(2017, 1, 30), datetime.date(2017, 2, 1)], 'NASDAQ:DCIX': [datetime.date(2017, 1, 18), datetime.date(2017, 2, 1), datetime.date(2017, 2, 13)], 'NASDAQ:ZAIS': [datetime.date(2017, 2, 3), datetime.date(2017, 2, 14), datetime.date(2017, 2, 15), datetime.date(2017, 2, 17), datetime.date(2017, 2, 23)], 'NASDAQ:HTGM': [datetime.date(2017, 2, 27)], 'NASDAQ:APRI': [datetime.date(2017, 1, 18), datetime.date(2017, 1, 20), datetime.date(2017, 2, 10)], 'NYSE:NMM': [datetime.date(2017, 2, 14)], 'NASDAQ:SHLD': [datetime.date(2017, 2, 14)], 'NASDAQ:DCTH': [datetime.date(2017, 1, 17), datetime.date(2017, 1, 24), datetime.date(2017, 1, 27), datetime.date(2017, 2, 16), datetime.date(2017, 2, 17), datetime.date(2017, 2, 22)], 'NASDAQ:CIDM': [datetime.date(2017, 1, 17), datetime.date(2017, 2, 15), datetime.date(2017, 3, 1)], 'NASDAQ:ANY': [datetime.date(2017, 1, 19), datetime.date(2017, 1, 20), datetime.date(2017, 1, 30), datetime.date(2017, 2, 9)], 'NASDAQ:XGTI': [datetime.date(2017, 2, 8)], 'NASDAQ:ADHD': [datetime.date(2017, 2, 8)], 'NASDAQ:GALT': [datetime.date(2017, 2, 6), datetime.date(2017, 2, 7), datetime.date(2017, 2, 14), datetime.date(2017, 2, 21), datetime.date(2017, 2, 22)], 'NYSE:IMN': [datetime.date(2017, 2, 17)], 'NASDAQ:PSIX': [datetime.date(2017, 2, 6), datetime.date(2017, 2, 16), datetime.date(2017, 2, 17)], 'NASDAQ:WCST': [datetime.date(2017, 2, 8)], 'NASDAQ:GENE': [datetime.date(2017, 2, 10)], 'NASDAQ:ATEC': [datetime.date(2017, 3, 3)], 'NYSE:EMG': [datetime.date(2017, 1, 19)], 'NASDAQ:OREX': [datetime.date(2017, 1, 25)], 'NASDAQ:MBOT': [datetime.date(2017, 2, 22), datetime.date(2017, 2, 28)], 'NASDAQ:ROKA': [datetime.date(2017, 2, 3)], 'NYSEMKT:BGI': [datetime.date(2017, 1, 23), datetime.date(2017, 1, 24)], 'NASDAQ:NETE': [datetime.date(2017, 1, 19), datetime.date(2017, 2, 8)], 'NASDAQ:LBIX': [datetime.date(2017, 2, 17)], 'NASDAQ:SIFY': [datetime.date(2017, 1, 20)], 'NASDAQ:CXRX': [datetime.date(2017, 2, 9)], 'NASDAQ:AEHR': [datetime.date(2017, 2, 8), datetime.date(2017, 2, 21)], 'NYSEMKT:CVM': [datetime.date(2017, 1, 17), datetime.date(2017, 2, 9), datetime.date(2017, 2, 17)], 'NASDAQ:WINT': [datetime.date(2017, 2, 2)], 'NASDAQ:MRNS': [datetime.date(2017, 1, 17), datetime.date(2017, 1, 23)], 'NASDAQ:IDXG': [datetime.date(2017, 2, 2)], 'NASDAQ:BIOS': [datetime.date(2017, 3, 3)], 'NASDAQ:NOVN': [datetime.date(2017, 2, 2), datetime.date(2017, 2, 15), datetime.date(2017, 2, 27)], 'NASDAQ:GIGA': [datetime.date(2017, 1, 23)], 'NASDAQ:CRBP': [datetime.date(2017, 2, 28)], 'NASDAQ:SSH': [datetime.date(2017, 1, 23)], 'NYSE:SSI': [datetime.date(2017, 3, 2)], 'NASDAQ:IFON': [datetime.date(2017, 2, 17)], 'NASDAQ:INVE': [datetime.date(2017, 2, 8)], 'NYSEMKT:AIRI': [datetime.date(2017, 1, 30)], 'NASDAQ:TSRO': [datetime.date(2017, 2, 8)], 'NASDAQ:IPHS': [datetime.date(2017, 2, 22)], 'NASDAQ:KPTI': [datetime.date(2017, 3, 2)], 'NASDAQ:COYN': [datetime.date(2017, 1, 25), datetime.date(2017, 2, 14)], 'NASDAQ:NXTD': [datetime.date(2017, 2, 2)], 'NASDAQ:CLRB': [datetime.date(2017, 1, 24)], 'NYSE:EGY': [datetime.date(2017, 1, 26)], 'NASDAQ:OPXA': [datetime.date(2017, 1, 24), datetime.date(2017, 2, 14)], 'NYSE:GLF': [datetime.date(2017, 1, 26)], 'NYSEMKT:GMO': [datetime.date(2017, 1, 20), datetime.date(2017, 1, 26)], 'NASDAQ:WASH': [datetime.date(2017, 1, 26)], 'NASDAQ:LPTH': [datetime.date(2017, 2, 14)], 'NYSEMKT:IGC': [datetime.date(2017, 2, 21), datetime.date(2017, 2, 22), datetime.date(2017, 2, 23), datetime.date(2017, 2, 28)], 'NASDAQ:OPGN': [datetime.date(2017, 2, 14)], 'NASDAQ:SKYS': [datetime.date(2017, 2, 9)], 'NASDAQ:CYCC': [datetime.date(2017, 2, 23)], 'NASDAQ:NFEC': [datetime.date(2017, 1, 19), datetime.date(2017, 2, 28)], 'NASDAQ:TENX': [datetime.date(2017, 1, 31), datetime.date(2017, 2, 1), datetime.date(2017, 2, 2)], 'NASDAQ:DRYS': [datetime.date(2017, 1, 24), datetime.date(2017, 1, 30), datetime.date(2017, 2, 1), datetime.date(2017, 2, 3), datetime.date(2017, 2, 13), datetime.date(2017, 2, 28)], 'NASDAQ:RNVA': [datetime.date(2017, 2, 17), datetime.date(2017, 2, 24), datetime.date(2017, 3, 2), datetime.date(2017, 3, 3)], 'NASDAQ:EDIT': [datetime.date(2017, 2, 15)], 'NASDAQ:KTOV': [datetime.date(2017, 2, 9), datetime.date(2017, 2, 27)], 'NYSEMKT:CVRS': [datetime.date(2017, 1, 23), datetime.date(2017, 2, 28), datetime.date(2017, 3, 3)], 'NYSEMKT:PIP': [datetime.date(2017, 2, 10)], 'NASDAQ:PULM': [datetime.date(2017, 1, 17), datetime.date(2017, 1, 19), datetime.date(2017, 1, 20), datetime.date(2017, 1, 24), datetime.date(2017, 1, 25), datetime.date(2017, 1, 26), datetime.date(2017, 1, 27), datetime.date(2017, 1, 30), datetime.date(2017, 2, 1), datetime.date(2017, 2, 2), datetime.date(2017, 2, 6), datetime.date(2017, 2, 23), datetime.date(2017, 2, 27)], 'NASDAQ:MDCA': [datetime.date(2017, 2, 15)], 'NASDAQ:AMTX': [datetime.date(2017, 1, 19), datetime.date(2017, 1, 25)], 'NASDAQ:MFIN': [datetime.date(2017, 2, 3)], 'NASDAQ:AVEO': [datetime.date(2017, 2, 15)], 'NASDAQ:NAKD': [datetime.date(2017, 1, 19), datetime.date(2017, 1, 25), datetime.date(2017, 2, 2), datetime.date(2017, 2, 6), datetime.date(2017, 2, 7), datetime.date(2017, 2, 14), datetime.date(2017, 2, 24)], 'NASDAQ:COOL': [datetime.date(2017, 2, 23), datetime.date(2017, 2, 24)], 'NASDAQ:CERU': [datetime.date(2017, 2, 3)], 'NASDAQ:GMAN': [datetime.date(2017, 1, 24), datetime.date(2017, 2, 13)], 'NASDAQ:CRTN': [datetime.date(2017, 1, 20), datetime.date(2017, 1, 23)], 'NASDAQ:MOSY': [datetime.date(2017, 1, 25), datetime.date(2017, 2, 9), datetime.date(2017, 2, 16), datetime.date(2017, 2, 17), datetime.date(2017, 2, 28)], 'NASDAQ:IKGH': [datetime.date(2017, 3, 2)], 'NASDAQ:ASYS': [datetime.date(2017, 1, 24)], 'NASDAQ:GNVC': [datetime.date(2017, 1, 18)], 'NASDAQ:VNR': [datetime.date(2017, 2, 2), datetime.date(2017, 2, 7), datetime.date(2017, 2, 9)], 'NASDAQ:BLIN': [datetime.date(2017, 2, 24), datetime.date(2017, 3, 2)], 'NASDAQ:LIFE': [datetime.date(2017, 1, 18)], 'NASDAQ:RKDA': [datetime.date(2017, 2, 15)], 'NASDAQ:LTRX': [datetime.date(2017, 1, 27)], 'NASDAQ:FCEL': [datetime.date(2017, 1, 26)], 'NASDAQ:PHMD': [datetime.date(2017, 2, 2), datetime.date(2017, 2, 3), datetime.date(2017, 2, 14)], 'NASDAQ:PACB': [datetime.date(2017, 2, 3)], 'NASDAQ:MDGS': [datetime.date(2017, 1, 23), datetime.date(2017, 2, 15)], 'NASDAQ:RTK': [datetime.date(2017, 3, 1)], 'NASDAQ:EGHT': [datetime.date(2017, 2, 24)], 'NASDAQ:CSBR': [datetime.date(2017, 1, 27), datetime.date(2017, 3, 1)], 'NASDAQ:CATB': [datetime.date(2017, 1, 19)], 'NASDAQ:DFFN': [datetime.date(2017, 1, 20), datetime.date(2017, 2, 2), datetime.date(2017, 2, 3)], 'NASDAQ:MARA': [datetime.date(2017, 1, 24)], 'NASDAQ:RXII': [datetime.date(2017, 2, 8)], 'NYSE:STNG': [datetime.date(2017, 2, 13)], 'NYSEMKT:ATNM': [datetime.date(2017, 2, 3), datetime.date(2017, 2, 9)], 'NASDAQ:TRIL': [datetime.date(2017, 2, 24)], 'NYSEMKT:ELMD': [datetime.date(2017, 2, 13)], 'NYSEMKT:AXN': [datetime.date(2017, 1, 25)], 'NASDAQ:IMMY': [datetime.date(2017, 2, 16)], 'NASDAQ:DELT': [datetime.date(2017, 1, 19), datetime.date(2017, 1, 20), datetime.date(2017, 1, 23), datetime.date(2017, 1, 26), datetime.date(2017, 2, 14)], 'NASDAQ:CUR': [datetime.date(2017, 2, 16), datetime.date(2017, 2, 28)], 'NASDAQ:DRAM': [datetime.date(2017, 2, 10)], 'NYSEMKT:XTNT': [datetime.date(2017, 1, 25), datetime.date(2017, 2, 27), datetime.date(2017, 2, 28)], 'NASDAQ:PLKI': [datetime.date(2017, 2, 13)], 'NYSEMKT:PLX': [datetime.date(2017, 2, 13)], 'NYSE:NQ': [datetime.date(2017, 2, 27)], 'NYSEMKT:MCF': [datetime.date(2017, 1, 26)], 'NASDAQ:ORIG': [datetime.date(2017, 2, 23)], 'NASDAQ:AXGN': [datetime.date(2017, 1, 23)], 'NASDAQ:RLJE': [datetime.date(2017, 1, 18)], 'NASDAQ:CBIO': [datetime.date(2017, 2, 7), datetime.date(2017, 2, 13), datetime.date(2017, 2, 22)], 'NYSEMKT:VSR': [datetime.date(2017, 2, 14)], 'NYSEMKT:RVP': [datetime.date(2017, 1, 20)], 'NASDAQ:IRG': [datetime.date(2017, 1, 20)], 'NYSE:DGI': [datetime.date(2017, 2, 17)], 'NASDAQ:VRAY': [datetime.date(2017, 1, 19), datetime.date(2017, 2, 27), datetime.date(2017, 3, 2)], 'NASDAQ:CALA': [datetime.date(2017, 1, 30), datetime.date(2017, 3, 2)], 'NASDAQ:FXCM': [datetime.date(2017, 2, 7)], 'NYSE:CBR': [datetime.date(2017, 2, 1), datetime.date(2017, 2, 3), datetime.date(2017, 2, 8)], 'NASDAQ:ESES': [datetime.date(2017, 1, 27), datetime.date(2017, 1, 30), datetime.date(2017, 2, 1), datetime.date(2017, 2, 2), datetime.date(2017, 2, 7)], 'NASDAQ:MNKD': [datetime.date(2017, 2, 2)], 'NYSE:XNY': [datetime.date(2017, 1, 23)], 'NASDAQ:CLSN': [datetime.date(2017, 2, 9)], 'NASDAQ:BSTG': [datetime.date(2017, 2, 16)], 'NASDAQ:PZRX': [datetime.date(2017, 2, 2)], 'NYSEMKT:MCZ': [datetime.date(2017, 2, 3), datetime.date(2017, 2, 7), datetime.date(2017, 2, 16), datetime.date(2017, 2, 24), datetime.date(2017, 2, 27)], 'NASDAQ:MYOS': [datetime.date(2017, 1, 26), datetime.date(2017, 2, 3), datetime.date(2017, 2, 6), datetime.date(2017, 2, 7), datetime.date(2017, 2, 13), datetime.date(2017, 2, 23), datetime.date(2017, 3, 1), datetime.date(2017, 3, 2)], 'NASDAQ:ORPN': [datetime.date(2017, 1, 26)], 'NASDAQ:BLPH': [datetime.date(2017, 2, 14), datetime.date(2017, 2, 21), datetime.date(2017, 2, 22), datetime.date(2017, 2, 28)], 'NYSEMKT:NSPR': [datetime.date(2017, 2, 14)], 'NASDAQ:JAKK': [datetime.date(2017, 2, 10)], 'NASDAQ:USAK': [datetime.date(2017, 2, 8)], 'NASDAQ:GEVO': [datetime.date(2017, 2, 23)], 'NYSEMKT:NAVB': [datetime.date(2017, 2, 13), datetime.date(2017, 2, 23), datetime.date(2017, 3, 3)], 'NASDAQ:AKER': [datetime.date(2017, 2, 16)], 'NASDAQ:NVFY': [datetime.date(2017, 1, 19), datetime.date(2017, 2, 21), datetime.date(2017, 2, 22), datetime.date(2017, 2, 23), datetime.date(2017, 2, 28), datetime.date(2017, 3, 1)], 'NASDAQ:MBRX': [datetime.date(2017, 2, 1), datetime.date(2017, 2, 7), datetime.date(2017, 2, 8), datetime.date(2017, 2, 15)], 'NASDAQ:NIHD': [datetime.date(2017, 2, 1)], 'NASDAQ:ACUR': [datetime.date(2017, 2, 7), datetime.date(2017, 2, 8), datetime.date(2017, 2, 9), datetime.date(2017, 2, 10)], 'NASDAQ:EIGI': [datetime.date(2017, 2, 16)], 'NASDAQ:SINO': [datetime.date(2017, 2, 13)], 'NASDAQ:CBMX': [datetime.date(2017, 2, 2)], 'NASDAQ:VRML': [datetime.date(2017, 2, 14), datetime.date(2017, 2, 15), datetime.date(2017, 2, 21)], 'NASDAQ:RGSE': [datetime.date(2017, 1, 19), datetime.date(2017, 1, 27), datetime.date(2017, 1, 30), datetime.date(2017, 2, 1), datetime.date(2017, 2, 3), datetime.date(2017, 2, 6), datetime.date(2017, 2, 17), datetime.date(2017, 2, 23)], 'NASDAQ:CENX': [datetime.date(2017, 2, 24)], 'NASDAQ:RELV': [datetime.date(2017, 2, 14)], 'NASDAQ:SYRX': [datetime.date(2017, 1, 19), datetime.date(2017, 1, 30)], 'NASDAQ:INFI': [datetime.date(2017, 3, 2)], 'NASDAQ:PTX': [datetime.date(2017, 1, 17), datetime.date(2017, 2, 3)], 'NYSE:SGY': [datetime.date(2017, 1, 20), datetime.date(2017, 1, 23), datetime.date(2017, 2, 24)], 'NYSEMKT:TEUM': [datetime.date(2017, 2, 23), datetime.date(2017, 2, 27), datetime.date(2017, 3, 3)], 'NASDAQ:CETX': [datetime.date(2017, 3, 1)], 'NYSE:CRR': [datetime.date(2017, 1, 26)], 'NASDAQ:UNXL': [datetime.date(2017, 1, 18)], 'NYSE:EMES': [datetime.date(2017, 2, 27)], 'NASDAQ:GRVY': [datetime.date(2017, 1, 18), datetime.date(2017, 1, 19), datetime.date(2017, 2, 22)], 'NASDAQ:MSDI': [datetime.date(2017, 1, 26)], 'NASDAQ:SKLN': [datetime.date(2017, 1, 17), datetime.date(2017, 3, 1)], 'NASDAQ:PPHM': [datetime.date(2017, 2, 17)], 'NASDAQ:RCON': [datetime.date(2017, 2, 6)], 'NYSEMKT:IMUC': [datetime.date(2017, 2, 14)], 'NASDAQ:AXTI': [datetime.date(2017, 2, 23)], 'NASDAQ:ANTH': [datetime.date(2017, 1, 27), datetime.date(2017, 2, 22)], 'NASDAQ:AHPI': [datetime.date(2017, 3, 3)], 'NASDAQ:REXX': [datetime.date(2017, 1, 17), datetime.date(2017, 1, 18), datetime.date(2017, 2, 8)], 'NASDAQ:TVIA': [datetime.date(2017, 1, 19), datetime.date(2017, 2, 9), datetime.date(2017, 2, 15)], 'NASDAQ:RLOG': [datetime.date(2017, 2, 14)], 'NASDAQ:TRCH': [datetime.date(2017, 2, 27)], 'NYSE:GRAM': [datetime.date(2017, 2, 28)], 'NYSEMKT:TPLM': [datetime.date(2017, 1, 17), datetime.date(2017, 1, 20), datetime.date(2017, 1, 31)], 'NYSEMKT:CANF': [datetime.date(2017, 1, 18)], 'NASDAQ:TBIO': [datetime.date(2017, 1, 20), datetime.date(2017, 1, 26), datetime.date(2017, 1, 31), datetime.date(2017, 2, 3), datetime.date(2017, 2, 9)], 'NASDAQ:EVOK': [datetime.date(2017, 1, 30)], 'NASDAQ:SBOT': [datetime.date(2017, 3, 3)], 'NASDAQ:ACST': [datetime.date(2017, 1, 23)], 'NYSEMKT:ENRJ': [datetime.date(2017, 1, 20), datetime.date(2017, 1, 23), datetime.date(2017, 1, 26), datetime.date(2017, 2, 21)], 'NASDAQ:SBBP': [datetime.date(2017, 3, 2)], 'NASDAQ:SGNL': [datetime.date(2017, 2, 7), datetime.date(2017, 2, 13)], 'NYSEMKT:DSS': [datetime.date(2017, 2, 3), datetime.date(2017, 2, 7)], 'NASDAQ:EVAR': [datetime.date(2017, 1, 20)], 'NASDAQ:YECO': [datetime.date(2017, 1, 17)], 'NASDAQ:LNTH': [datetime.date(2017, 2, 22)], 'NYSE:BCEI': [datetime.date(2017, 1, 17)], 'NASDAQ:DVAX': [datetime.date(2017, 3, 1)], 'NASDAQ:PRZM': [datetime.date(2017, 2, 10)], 'NASDAQ:INNL': [datetime.date(2017, 1, 18), datetime.date(2017, 3, 1)], 'NASDAQ:JAGX': [datetime.date(2017, 1, 31), datetime.date(2017, 2, 1), datetime.date(2017, 2, 17), datetime.date(2017, 2, 21), datetime.date(2017, 2, 24), datetime.date(2017, 2, 27)], 'NASDAQ:LPSN': [datetime.date(2017, 2, 9)], 'NASDAQ:ARGS': [datetime.date(2017, 3, 2)], 'NASDAQ:CHCI': [datetime.date(2017, 1, 18)], 'NASDAQ:INVT': [datetime.date(2017, 1, 27), datetime.date(2017, 1, 30), datetime.date(2017, 1, 31), datetime.date(2017, 2, 15)], 'NYSEMKT:HEB': [datetime.date(2017, 2, 13)], 'NASDAQ:WRLD': [datetime.date(2017, 1, 26)], 'NYSE:SXE': [datetime.date(2017, 2, 16)], 'NASDAQ:SCON': [datetime.date(2017, 2, 16)], 'NASDAQ:PME': [datetime.date(2017, 1, 17)], 'NASDAQ:BONT': [datetime.date(2017, 2, 9)], 'NASDAQ:BASI': [datetime.date(2017, 1, 19), datetime.date(2017, 2, 14), datetime.date(2017, 2, 22), datetime.date(2017, 2, 27), datetime.date(2017, 2, 28)], 'NASDAQ:AEZS': [datetime.date(2017, 1, 17), datetime.date(2017, 1, 31)], 'NASDAQ:OCRX': [datetime.date(2017, 2, 7), datetime.date(2017, 2, 8), datetime.date(2017, 2, 9)], 'NASDAQ:FNCX': [datetime.date(2017, 1, 25), datetime.date(2017, 2, 16), datetime.date(2017, 2, 23)], 'NYSE:FAC': [datetime.date(2017, 1, 27)], 'NASDAQ:MEMP': [datetime.date(2017, 1, 17), datetime.date(2017, 1, 25), datetime.date(2017, 1, 26), datetime.date(2017, 1, 27), datetime.date(2017, 2, 15), datetime.date(2017, 2, 16)], 'NASDAQ:WKHS': [datetime.date(2017, 1, 26)], 'NASDAQ:EYES': [datetime.date(2017, 2, 7), datetime.date(2017, 2, 10)], 'NASDAQ:SEV': [datetime.date(2017, 2, 3)], 'NASDAQ:ALQA': [datetime.date(2017, 3, 2)], 'NYSEMKT:NAK': [datetime.date(2017, 2, 21), datetime.date(2017, 2, 22), datetime.date(2017, 2, 23)], 'NASDAQ:VIVO': [datetime.date(2017, 1, 25)], 'NASDAQ:VNRAP': [datetime.date(2017, 1, 25), datetime.date(2017, 2, 2), datetime.date(2017, 2, 3)], 'NASDAQ:STML': [datetime.date(2017, 2, 2)], 'NASDAQ:PRIM': [datetime.date(2017, 2, 28)], 'NYSE:OB': [datetime.date(2017, 1, 18)], 'NASDAQ:BCLI': [datetime.date(2017, 2, 1)], 'NASDAQ:KBSF': [datetime.date(2017, 2, 9), datetime.date(2017, 2, 10), datetime.date(2017, 2, 15), datetime.date(2017, 2, 16), datetime.date(2017, 2, 22), datetime.date(2017, 2, 23)], 'NASDAQ:BNTC': [datetime.date(2017, 1, 17), datetime.date(2017, 2, 2), datetime.date(2017, 2, 3), datetime.date(2017, 2, 7), datetime.date(2017, 2, 15), datetime.date(2017, 2, 16)], 'NYSE:CHKR': [datetime.date(2017, 1, 25)], 'NYSEMKT:APHB': [datetime.date(2017, 2, 8)], 'NYSEMKT:TPHS': [datetime.date(2017, 2, 28)], 'NASDAQ:STAF': [datetime.date(2017, 2, 17)], 'NYSE:BNED': [datetime.date(2017, 2, 28)], 'NASDAQ:SNMX': [datetime.date(2017, 2, 27)], 'NYSEMKT:SSN': [datetime.date(2017, 2, 21)], 'NASDAQ:CRDS': [datetime.date(2017, 1, 19), datetime.date(2017, 2, 13)], 'NASDAQ:NVAX': [datetime.date(2017, 2, 15)], 'NASDAQ:FATE': [datetime.date(2017, 2, 17)], 'NASDAQ:MBVX': [datetime.date(2017, 2, 23)], 'NASDAQ:BIOL': [datetime.date(2017, 1, 17)], 'NASDAQ:TOPS': [datetime.date(2017, 2, 16), datetime.date(2017, 2, 24)], 'NASDAQ:URRE': [datetime.date(2017, 1, 19)], 'NASDAQ:APOP': [datetime.date(2017, 2, 2), datetime.date(2017, 2, 8), datetime.date(2017, 2, 14), datetime.date(2017, 2, 21)]}


figsize(10,4)

hit_r_tot, miss_r_tot = 0, 0
hit_k_tot, miss_k_tot = 0, 0
hit_g_tot, miss_g_tot = 0, 0

for symb in noticable_dic:
    for day in noticable_dic[symb]:
        df = screener.GetIntradayMinutData(symb, day)
        if isinstance(df, pd.DataFrame):
            #print(df.head())
            P = df['CLOSE'].values
            V = df['VOLUME'].values
            T = df.index.values
            
            N = len(T)

            dP = P[1:]-P[:-1]
            dV = V[1:]-V[:-1]
            dT = T[1:]-T[:-1]
            
            N_avg = 5
            P_avg = np.zeros(N)
            for i in range(N):
                P_avg[i] = np.sum(P[max(i+1-N_avg, 0):i+1])/(i+1-max(i+1-N_avg, 0))

            N_avg = 10
            V_avg = np.zeros(N)
            for i in range(N):
                V_avg[i] = np.sum(V[max(i+1-N_avg, 0):i+1])/(i+1-max(i+1-N_avg, 0))

            dPV = P[1:]*V[1:]-P[:-1]*V[:-1]
            VdP = V[1:]*dP
            intVdP = np.zeros(len(VdP))
            intVdP[0] = VdP[0]
            for i in range(1, len(VdP)):
                intVdP[i] = intVdP[i-1]+VdP[i]
            intVdP_rel = intVdP/np.abs(intVdP).max()

            dintVdP_rel = intVdP_rel[1:] - intVdP_rel[:-1]
            #dP_divV = dP/V[1:]
            #dP_divV[1:] *= (np.abs(dintVdP_rel) > np.abs(dintVdP_rel).max()/10)
            #dP_divV[0] = 0.0

            N_avg = 20
            intVdP_avg = np.zeros(N-1)
            for i in range(N-1):
                intVdP_avg[i] = np.sum(intVdP[max(i+1-N_avg, 0):i+1])/(i+1-max(i+1-N_avg, 0))
            intVdP_rel_avg = intVdP_avg/np.abs(intVdP).max()
            
            
            plot(T, P/P.max(), 'b', lw=1.0)
            plot(T, P_avg/P.max(), 'b', lw=0.4)
            #plot(T, (P-P.min())/(P.max()-P.min()), 'b', lw=1.0)
            #bar(T, V/V.max(), width=0.0002, color='b', linewidth=0.00001)
            plot(T, V/V.max(), color='g', linewidth=0.4)
            plot(T, V_avg/V.max(), color='g', linewidth=1.0)
            plot(T[1:], intVdP_rel, 'r')
            plot(T[1:], intVdP_rel_avg, 'r', linewidth=0.4)
            #plot(T[1:], dP_divV/np.abs(dP_divV).max(), 'k')
            
            i_up = -10
            i_up_r = -30
            N = len(T)
            hit_k , miss_k = 0, 0
            hit_g , miss_g = 0, 0
            hit_r,  miss_r = 0, 0
            for i in range(2, N-10):
                ii = i-1
                if np.abs(P[:i+1]).max() - np.abs(P[:i+1]).min()>np.abs(P[:i+1]).min()*0.05 \
                        and P[i]*V[i]>50.0e3 and 1.0<P[i]<10 and V[i]>0.7*V[i-1]:
                    X = np.abs(intVdP_rel[:max(ii-2,2)]*(intVdP_rel[:max(ii-2,2)]>0.0)).max()
                    if (i-i_up_r)>25 and intVdP_rel[ii]>2.0*X:# and intVdP_rel[ii]<10.0*X:
                        axvline(T[i], color='r', linestyle='-.', linewidth=2)
                        i_up_r = i
                        j0 = i
                        j1 = min(j0+30, N)
                        if (P[j0:j1].max()-P[j0])/P[j0]>0.03:
                            hit_r += 1
                        else:
                            miss_r += 1
            for i in range(2, len(dintVdP_rel)):
                if np.abs(P[:i]).max() - np.abs(P[:i]).min()>np.abs(P[:i]).min()/10:
                    if dintVdP_rel[i]<-np.abs(dintVdP_rel[:i]).max()/5 and \
                        (P[i+2]-P_avg[i+2])/P_avg[i+2]<-0.05:
                        if np.argmax(intVdP_rel[:i])<i-5:
                            axvline(T[i+2], color='k', linestyle='-.')
                            j0 = i+2
                            j1 = min(j0+10, N)
                            if (P[j0:j1].max()-P[j0])/P[j0]>0.03:
                                hit_k += 1
                            else:
                                miss_k += 1
            for i in range(2, len(intVdP_rel)):
                if (P[max(i-10,0):i+1].max()-P[max(i-10,0):i+1].min())/P[max(i-10,0):i+1].min()>0.05 and np.argmax(P[:i+1])>i-10:
                    ind_max = np.argmax(intVdP_rel[:i])
                    if max(i-10,2)<ind_max<i-1 and i-i_up>5:
                        if np.argmin(intVdP_rel[ind_max:i+1])<(i+1-ind_max) and np.argmax(intVdP_rel[:i+1])==i:
                            i_up = i
                            axvline(T[i_up], color='g', linestyle=':')
                            j0 = i_up
                            j1 = min(i_up+20, N)
                            if (P[j0:j1].max()-P[j0])/P[j0]>0.04:
                                hit_g += 1
                            else:
                                miss_g += 1
                        
                    elif i-i_up>5 and np.argmax(intVdP_rel[:i+1])==i and intVdP_rel[i-2]>0 and dintVdP_rel[i-1]>0.2*dintVdP_rel[i-2]:
                        i_up = i
                        axvline(T[i_up], color='g', linestyle='-.')
                        j0 = i_up
                        j1 = min(i_up+20, N)
                        if (P[j0:j1].max()-P[j0])/P[j0]>0.04:
                            hit_g += 1
                        else:
                            miss_g += 1
                        
                        
            
            title('{} - {}'.format(symb, day))
            show()
            print('price range:', P.min(), P.max())
            print('volume avg: {},   max :{}   , PV_max: {}'.format(V.mean()/1.0e3, V.max()/1.0e3, (P*V).max()/1.0e3))
            print('r:  hits: ', hit_r, '  misses: ', miss_r)
            print('k:  hits: ', hit_k, '  misses: ', miss_k)
            print('g:  hits: ', hit_g, '  misses: ', miss_g)
            hit_r_tot += hit_r
            miss_r_tot += miss_r
            hit_k_tot += hit_k
            miss_k_tot += miss_k
            hit_g_tot += hit_g
            miss_g_tot += miss_g
print('r:  hits total: ', hit_r_tot, '  misses total: ', miss_r_tot, 'ratio:', hit_r_tot/miss_r_tot)
print('k:  hits total: ', hit_k_tot, '  misses total: ', miss_k_tot, 'ratio:', hit_k_tot/miss_k_tot)
print('g:  hits total: ', hit_g_tot, '  misses total: ', miss_g_tot,  'ratio:', hit_g_tot/miss_g_tot)


In [ ]:
%pylab inline

from Finance.Finance import *
import datetime
import numpy as np
import pandas as pd

screener = StockScreener()

# 20%
#noticable_dic = {'NASDAQ:ZSAN': [datetime.date(2017, 2, 13), datetime.date(2017, 2, 24)], 'NASDAQ:JAGX': [datetime.date(2017, 2, 27)], 'NASDAQ:TBIO': [datetime.date(2017, 2, 9)], 'NYSEMKT:ESNC': [datetime.date(2017, 3, 2)], 'NASDAQ:VNRCP': [datetime.date(2017, 2, 2)], 'NASDAQ:PHMD': [datetime.date(2017, 2, 14)], 'NASDAQ:PSIX': [datetime.date(2017, 2, 16)], 'NASDAQ:GLBS': [datetime.date(2017, 1, 17), datetime.date(2017, 1, 18)], 'NASDAQ:CBIO': [datetime.date(2017, 2, 13)], 'NASDAQ:QBAK': [datetime.date(2017, 2, 15)], 'NYSEMKT:ENRJ': [datetime.date(2017, 1, 20)], 'NASDAQ:OPGN': [datetime.date(2017, 2, 14)], 'NASDAQ:SKYS': [datetime.date(2017, 2, 9)], 'NYSEMKT:DSS': [datetime.date(2017, 2, 3)], 'NASDAQ:NFEC': [datetime.date(2017, 1, 19), datetime.date(2017, 2, 28)], 'NASDAQ:EVAR': [datetime.date(2017, 1, 20)], 'NASDAQ:VRAY': [datetime.date(2017, 2, 27)], 'NASDAQ:VNRBP': [datetime.date(2017, 2, 2), datetime.date(2017, 2, 3)], 'NASDAQ:FXCM': [datetime.date(2017, 2, 7)], 'NASDAQ:RNVA': [datetime.date(2017, 2, 17), datetime.date(2017, 3, 3)], 'NASDAQ:ETRM': [datetime.date(2017, 1, 20)], 'NASDAQ:ESES': [datetime.date(2017, 1, 27), datetime.date(2017, 1, 30)], 'NASDAQ:VNR': [datetime.date(2017, 2, 2), datetime.date(2017, 2, 7)], 'NASDAQ:KBSF': [datetime.date(2017, 2, 9), datetime.date(2017, 2, 10), datetime.date(2017, 2, 22)], 'NYSE:XNY': [datetime.date(2017, 1, 23)], 'NASDAQ:KTOV': [datetime.date(2017, 2, 9)], 'NASDAQ:DFFN': [datetime.date(2017, 2, 2), datetime.date(2017, 2, 3)], 'NYSEMKT:PIP': [datetime.date(2017, 2, 6)], 'NASDAQ:INAP': [datetime.date(2017, 2, 22)], 'NASDAQ:MEMP': [datetime.date(2017, 1, 17), datetime.date(2017, 2, 16)], 'NASDAQ:PULM': [datetime.date(2017, 1, 17), datetime.date(2017, 1, 26), datetime.date(2017, 2, 1), datetime.date(2017, 2, 2)], 'NASDAQ:MYOS': [datetime.date(2017, 2, 3), datetime.date(2017, 2, 7)], 'NASDAQ:AMTX': [datetime.date(2017, 1, 19)], 'NASDAQ:ZAIS': [datetime.date(2017, 2, 14)], 'NASDAQ:CHCI': [datetime.date(2017, 1, 18)], 'NASDAQ:VNRAP': [datetime.date(2017, 2, 2)], 'NASDAQ:HTGM': [datetime.date(2017, 2, 27)], 'NYSEMKT:MCZ': [datetime.date(2017, 2, 3)], 'NASDAQ:MBRX': [datetime.date(2017, 2, 1)], 'NASDAQ:CERU': [datetime.date(2017, 2, 3)], 'NYSEMKT:BGI': [datetime.date(2017, 1, 24)], 'NYSE:EMG': [datetime.date(2017, 1, 19)], 'NASDAQ:VRML': [datetime.date(2017, 2, 21)], 'NASDAQ:MOSY': [datetime.date(2017, 2, 17)], 'NASDAQ:NVFY': [datetime.date(2017, 2, 21), datetime.date(2017, 2, 23)], 'NASDAQ:BASI': [datetime.date(2017, 1, 19), datetime.date(2017, 2, 14), datetime.date(2017, 2, 27), datetime.date(2017, 2, 28)], 'NASDAQ:XGTI': [datetime.date(2017, 2, 8)], 'NASDAQ:OCRX': [datetime.date(2017, 2, 8)], 'NASDAQ:RLOG': [datetime.date(2017, 2, 14)], 'NASDAQ:GALT': [datetime.date(2017, 2, 22)], 'NASDAQ:FNCX': [datetime.date(2017, 1, 25), datetime.date(2017, 2, 23)], 'NASDAQ:SGNL': [datetime.date(2017, 2, 7)], 'NASDAQ:WCST': [datetime.date(2017, 2, 8)], 'NYSEMKT:IGC': [datetime.date(2017, 2, 22)], 'NASDAQ:RLJE': [datetime.date(2017, 1, 18)], 'NASDAQ:RGSE': [datetime.date(2017, 1, 30)], 'NASDAQ:FCEL': [datetime.date(2017, 1, 26)], 'NASDAQ:NETE': [datetime.date(2017, 1, 19), datetime.date(2017, 2, 8)], 'NASDAQ:SINO': [datetime.date(2017, 2, 13)], 'NASDAQ:LBIX': [datetime.date(2017, 2, 17)], 'NASDAQ:INFI': [datetime.date(2017, 3, 2)], 'NASDAQ:AEHR': [datetime.date(2017, 2, 8)], 'NASDAQ:BNTC': [datetime.date(2017, 1, 17), datetime.date(2017, 2, 2), datetime.date(2017, 2, 7)], 'NASDAQ:APRI': [datetime.date(2017, 1, 18)], 'NYSEMKT:TEUM': [datetime.date(2017, 2, 23)], 'NYSEMKT:APHB': [datetime.date(2017, 2, 8)], 'NASDAQ:MRNS': [datetime.date(2017, 1, 17)], 'NASDAQ:ACUR': [datetime.date(2017, 2, 7), datetime.date(2017, 2, 8), datetime.date(2017, 2, 10)], 'NASDAQ:NOVN': [datetime.date(2017, 1, 27), datetime.date(2017, 2, 15)], 'NYSEMKT:NAVB': [datetime.date(2017, 2, 23)], 'NASDAQ:NAKD': [datetime.date(2017, 2, 2), datetime.date(2017, 2, 6)], 'NASDAQ:MSDI': [datetime.date(2017, 1, 26)], 'NASDAQ:PPHM': [datetime.date(2017, 2, 17)], 'NYSE:SSI': [datetime.date(2017, 3, 2)], 'NASDAQ:CRDS': [datetime.date(2017, 1, 19), datetime.date(2017, 2, 13)], 'NASDAQ:DRYS': [datetime.date(2017, 1, 30)], 'NASDAQ:EYEG': [datetime.date(2017, 2, 21)], 'NASDAQ:SBBP': [datetime.date(2017, 3, 2)], 'NASDAQ:CIDM': [datetime.date(2017, 1, 17), datetime.date(2017, 2, 15)], 'NASDAQ:OPHC': [datetime.date(2017, 1, 17)], 'NASDAQ:APOP': [datetime.date(2017, 2, 2), datetime.date(2017, 2, 14)], 'NASDAQ:DELT': [datetime.date(2017, 1, 19)], 'NASDAQ:CUR': [datetime.date(2017, 2, 16)]}

# 10%
noticable_dic = {'NASDAQ:ZSAN': [datetime.date(2017, 1, 31), datetime.date(2017, 2, 13), datetime.date(2017, 2, 14), datetime.date(2017, 2, 15), datetime.date(2017, 2, 16), datetime.date(2017, 2, 24)], 'NYSEMKT:TGB': [datetime.date(2017, 2, 17)], 'NASDAQ:MNTX': [datetime.date(2017, 1, 17)], 'NYSEMKT:ESNC': [datetime.date(2017, 3, 2)], 'NASDAQ:VNRCP': [datetime.date(2017, 2, 2), datetime.date(2017, 2, 3)], 'NYSE:HGG': [datetime.date(2017, 2, 6), datetime.date(2017, 2, 8), datetime.date(2017, 2, 24)], 'NASDAQ:GLBS': [datetime.date(2017, 1, 18), datetime.date(2017, 1, 19), datetime.date(2017, 1, 23), datetime.date(2017, 1, 24), datetime.date(2017, 1, 26), datetime.date(2017, 2, 13)], 'NASDAQ:QBAK': [datetime.date(2017, 2, 15)], 'NASDAQ:PRKR': [datetime.date(2017, 1, 20)], 'NASDAQ:BIOC': [datetime.date(2017, 1, 19), datetime.date(2017, 2, 8), datetime.date(2017, 2, 10), datetime.date(2017, 2, 22)], 'NYSE:LXU': [datetime.date(2017, 2, 28)], 'NASDAQ:VNRBP': [datetime.date(2017, 2, 2), datetime.date(2017, 2, 3)], 'NASDAQ:DGLY': [datetime.date(2017, 1, 30)], 'NASDAQ:ETRM': [datetime.date(2017, 1, 20), datetime.date(2017, 1, 24), datetime.date(2017, 2, 1), datetime.date(2017, 2, 2)], 'NASDAQ:DXTR': [datetime.date(2017, 1, 17)], 'NASDAQ:INAP': [datetime.date(2017, 2, 22)], 'NASDAQ:OPHC': [datetime.date(2017, 1, 17)], 'NASDAQ:HMNY': [datetime.date(2017, 1, 23), datetime.date(2017, 1, 30), datetime.date(2017, 2, 1)], 'NASDAQ:DCIX': [datetime.date(2017, 1, 18), datetime.date(2017, 2, 1), datetime.date(2017, 2, 13)], 'NASDAQ:ZAIS': [datetime.date(2017, 2, 3), datetime.date(2017, 2, 14), datetime.date(2017, 2, 15), datetime.date(2017, 2, 17), datetime.date(2017, 2, 23)], 'NASDAQ:HTGM': [datetime.date(2017, 2, 27)], 'NASDAQ:APRI': [datetime.date(2017, 1, 18), datetime.date(2017, 1, 20), datetime.date(2017, 2, 10)], 'NYSE:NMM': [datetime.date(2017, 2, 14)], 'NASDAQ:SHLD': [datetime.date(2017, 2, 14)], 'NASDAQ:DCTH': [datetime.date(2017, 1, 17), datetime.date(2017, 1, 24), datetime.date(2017, 1, 27), datetime.date(2017, 2, 16), datetime.date(2017, 2, 17), datetime.date(2017, 2, 22)], 'NASDAQ:CIDM': [datetime.date(2017, 1, 17), datetime.date(2017, 2, 15), datetime.date(2017, 3, 1)], 'NASDAQ:ANY': [datetime.date(2017, 1, 19), datetime.date(2017, 1, 20), datetime.date(2017, 1, 30), datetime.date(2017, 2, 9)], 'NASDAQ:XGTI': [datetime.date(2017, 2, 8)], 'NASDAQ:ADHD': [datetime.date(2017, 2, 8)], 'NASDAQ:GALT': [datetime.date(2017, 2, 6), datetime.date(2017, 2, 7), datetime.date(2017, 2, 14), datetime.date(2017, 2, 21), datetime.date(2017, 2, 22)], 'NYSE:IMN': [datetime.date(2017, 2, 17)], 'NASDAQ:PSIX': [datetime.date(2017, 2, 6), datetime.date(2017, 2, 16), datetime.date(2017, 2, 17)], 'NASDAQ:WCST': [datetime.date(2017, 2, 8)], 'NASDAQ:GENE': [datetime.date(2017, 2, 10)], 'NASDAQ:ATEC': [datetime.date(2017, 3, 3)], 'NYSE:EMG': [datetime.date(2017, 1, 19)], 'NASDAQ:OREX': [datetime.date(2017, 1, 25)], 'NASDAQ:MBOT': [datetime.date(2017, 2, 22), datetime.date(2017, 2, 28)], 'NASDAQ:ROKA': [datetime.date(2017, 2, 3)], 'NYSEMKT:BGI': [datetime.date(2017, 1, 23), datetime.date(2017, 1, 24)], 'NASDAQ:NETE': [datetime.date(2017, 1, 19), datetime.date(2017, 2, 8)], 'NASDAQ:LBIX': [datetime.date(2017, 2, 17)], 'NASDAQ:SIFY': [datetime.date(2017, 1, 20)], 'NASDAQ:CXRX': [datetime.date(2017, 2, 9)], 'NASDAQ:AEHR': [datetime.date(2017, 2, 8), datetime.date(2017, 2, 21)], 'NYSEMKT:CVM': [datetime.date(2017, 1, 17), datetime.date(2017, 2, 9), datetime.date(2017, 2, 17)], 'NASDAQ:WINT': [datetime.date(2017, 2, 2)], 'NASDAQ:MRNS': [datetime.date(2017, 1, 17), datetime.date(2017, 1, 23)], 'NASDAQ:IDXG': [datetime.date(2017, 2, 2)], 'NASDAQ:BIOS': [datetime.date(2017, 3, 3)], 'NASDAQ:NOVN': [datetime.date(2017, 2, 2), datetime.date(2017, 2, 15), datetime.date(2017, 2, 27)], 'NASDAQ:GIGA': [datetime.date(2017, 1, 23)], 'NASDAQ:CRBP': [datetime.date(2017, 2, 28)], 'NASDAQ:SSH': [datetime.date(2017, 1, 23)], 'NYSE:SSI': [datetime.date(2017, 3, 2)], 'NASDAQ:IFON': [datetime.date(2017, 2, 17)], 'NASDAQ:INVE': [datetime.date(2017, 2, 8)], 'NYSEMKT:AIRI': [datetime.date(2017, 1, 30)], 'NASDAQ:TSRO': [datetime.date(2017, 2, 8)], 'NASDAQ:IPHS': [datetime.date(2017, 2, 22)], 'NASDAQ:KPTI': [datetime.date(2017, 3, 2)], 'NASDAQ:COYN': [datetime.date(2017, 1, 25), datetime.date(2017, 2, 14)], 'NASDAQ:NXTD': [datetime.date(2017, 2, 2)], 'NASDAQ:CLRB': [datetime.date(2017, 1, 24)], 'NYSE:EGY': [datetime.date(2017, 1, 26)], 'NASDAQ:OPXA': [datetime.date(2017, 1, 24), datetime.date(2017, 2, 14)], 'NYSE:GLF': [datetime.date(2017, 1, 26)], 'NYSEMKT:GMO': [datetime.date(2017, 1, 20), datetime.date(2017, 1, 26)], 'NASDAQ:WASH': [datetime.date(2017, 1, 26)], 'NASDAQ:LPTH': [datetime.date(2017, 2, 14)], 'NYSEMKT:IGC': [datetime.date(2017, 2, 21), datetime.date(2017, 2, 22), datetime.date(2017, 2, 23), datetime.date(2017, 2, 28)], 'NASDAQ:OPGN': [datetime.date(2017, 2, 14)], 'NASDAQ:SKYS': [datetime.date(2017, 2, 9)], 'NASDAQ:CYCC': [datetime.date(2017, 2, 23)], 'NASDAQ:NFEC': [datetime.date(2017, 1, 19), datetime.date(2017, 2, 28)], 'NASDAQ:TENX': [datetime.date(2017, 1, 31), datetime.date(2017, 2, 1), datetime.date(2017, 2, 2)], 'NASDAQ:DRYS': [datetime.date(2017, 1, 24), datetime.date(2017, 1, 30), datetime.date(2017, 2, 1), datetime.date(2017, 2, 3), datetime.date(2017, 2, 13), datetime.date(2017, 2, 28)], 'NASDAQ:RNVA': [datetime.date(2017, 2, 17), datetime.date(2017, 2, 24), datetime.date(2017, 3, 2), datetime.date(2017, 3, 3)], 'NASDAQ:EDIT': [datetime.date(2017, 2, 15)], 'NASDAQ:KTOV': [datetime.date(2017, 2, 9), datetime.date(2017, 2, 27)], 'NYSEMKT:CVRS': [datetime.date(2017, 1, 23), datetime.date(2017, 2, 28), datetime.date(2017, 3, 3)], 'NYSEMKT:PIP': [datetime.date(2017, 2, 10)], 'NASDAQ:PULM': [datetime.date(2017, 1, 17), datetime.date(2017, 1, 19), datetime.date(2017, 1, 20), datetime.date(2017, 1, 24), datetime.date(2017, 1, 25), datetime.date(2017, 1, 26), datetime.date(2017, 1, 27), datetime.date(2017, 1, 30), datetime.date(2017, 2, 1), datetime.date(2017, 2, 2), datetime.date(2017, 2, 6), datetime.date(2017, 2, 23), datetime.date(2017, 2, 27)], 'NASDAQ:MDCA': [datetime.date(2017, 2, 15)], 'NASDAQ:AMTX': [datetime.date(2017, 1, 19), datetime.date(2017, 1, 25)], 'NASDAQ:MFIN': [datetime.date(2017, 2, 3)], 'NASDAQ:AVEO': [datetime.date(2017, 2, 15)], 'NASDAQ:NAKD': [datetime.date(2017, 1, 19), datetime.date(2017, 1, 25), datetime.date(2017, 2, 2), datetime.date(2017, 2, 6), datetime.date(2017, 2, 7), datetime.date(2017, 2, 14), datetime.date(2017, 2, 24)], 'NASDAQ:COOL': [datetime.date(2017, 2, 23), datetime.date(2017, 2, 24)], 'NASDAQ:CERU': [datetime.date(2017, 2, 3)], 'NASDAQ:GMAN': [datetime.date(2017, 1, 24), datetime.date(2017, 2, 13)], 'NASDAQ:CRTN': [datetime.date(2017, 1, 20), datetime.date(2017, 1, 23)], 'NASDAQ:MOSY': [datetime.date(2017, 1, 25), datetime.date(2017, 2, 9), datetime.date(2017, 2, 16), datetime.date(2017, 2, 17), datetime.date(2017, 2, 28)], 'NASDAQ:IKGH': [datetime.date(2017, 3, 2)], 'NASDAQ:ASYS': [datetime.date(2017, 1, 24)], 'NASDAQ:GNVC': [datetime.date(2017, 1, 18)], 'NASDAQ:VNR': [datetime.date(2017, 2, 2), datetime.date(2017, 2, 7), datetime.date(2017, 2, 9)], 'NASDAQ:BLIN': [datetime.date(2017, 2, 24), datetime.date(2017, 3, 2)], 'NASDAQ:LIFE': [datetime.date(2017, 1, 18)], 'NASDAQ:RKDA': [datetime.date(2017, 2, 15)], 'NASDAQ:LTRX': [datetime.date(2017, 1, 27)], 'NASDAQ:FCEL': [datetime.date(2017, 1, 26)], 'NASDAQ:PHMD': [datetime.date(2017, 2, 2), datetime.date(2017, 2, 3), datetime.date(2017, 2, 14)], 'NASDAQ:PACB': [datetime.date(2017, 2, 3)], 'NASDAQ:MDGS': [datetime.date(2017, 1, 23), datetime.date(2017, 2, 15)], 'NASDAQ:RTK': [datetime.date(2017, 3, 1)], 'NASDAQ:EGHT': [datetime.date(2017, 2, 24)], 'NASDAQ:CSBR': [datetime.date(2017, 1, 27), datetime.date(2017, 3, 1)], 'NASDAQ:CATB': [datetime.date(2017, 1, 19)], 'NASDAQ:DFFN': [datetime.date(2017, 1, 20), datetime.date(2017, 2, 2), datetime.date(2017, 2, 3)], 'NASDAQ:MARA': [datetime.date(2017, 1, 24)], 'NASDAQ:RXII': [datetime.date(2017, 2, 8)], 'NYSE:STNG': [datetime.date(2017, 2, 13)], 'NYSEMKT:ATNM': [datetime.date(2017, 2, 3), datetime.date(2017, 2, 9)], 'NASDAQ:TRIL': [datetime.date(2017, 2, 24)], 'NYSEMKT:ELMD': [datetime.date(2017, 2, 13)], 'NYSEMKT:AXN': [datetime.date(2017, 1, 25)], 'NASDAQ:IMMY': [datetime.date(2017, 2, 16)], 'NASDAQ:DELT': [datetime.date(2017, 1, 19), datetime.date(2017, 1, 20), datetime.date(2017, 1, 23), datetime.date(2017, 1, 26), datetime.date(2017, 2, 14)], 'NASDAQ:CUR': [datetime.date(2017, 2, 16), datetime.date(2017, 2, 28)], 'NASDAQ:DRAM': [datetime.date(2017, 2, 10)], 'NYSEMKT:XTNT': [datetime.date(2017, 1, 25), datetime.date(2017, 2, 27), datetime.date(2017, 2, 28)], 'NASDAQ:PLKI': [datetime.date(2017, 2, 13)], 'NYSEMKT:PLX': [datetime.date(2017, 2, 13)], 'NYSE:NQ': [datetime.date(2017, 2, 27)], 'NYSEMKT:MCF': [datetime.date(2017, 1, 26)], 'NASDAQ:ORIG': [datetime.date(2017, 2, 23)], 'NASDAQ:AXGN': [datetime.date(2017, 1, 23)], 'NASDAQ:RLJE': [datetime.date(2017, 1, 18)], 'NASDAQ:CBIO': [datetime.date(2017, 2, 7), datetime.date(2017, 2, 13), datetime.date(2017, 2, 22)], 'NYSEMKT:VSR': [datetime.date(2017, 2, 14)], 'NYSEMKT:RVP': [datetime.date(2017, 1, 20)], 'NASDAQ:IRG': [datetime.date(2017, 1, 20)], 'NYSE:DGI': [datetime.date(2017, 2, 17)], 'NASDAQ:VRAY': [datetime.date(2017, 1, 19), datetime.date(2017, 2, 27), datetime.date(2017, 3, 2)], 'NASDAQ:CALA': [datetime.date(2017, 1, 30), datetime.date(2017, 3, 2)], 'NASDAQ:FXCM': [datetime.date(2017, 2, 7)], 'NYSE:CBR': [datetime.date(2017, 2, 1), datetime.date(2017, 2, 3), datetime.date(2017, 2, 8)], 'NASDAQ:ESES': [datetime.date(2017, 1, 27), datetime.date(2017, 1, 30), datetime.date(2017, 2, 1), datetime.date(2017, 2, 2), datetime.date(2017, 2, 7)], 'NASDAQ:MNKD': [datetime.date(2017, 2, 2)], 'NYSE:XNY': [datetime.date(2017, 1, 23)], 'NASDAQ:CLSN': [datetime.date(2017, 2, 9)], 'NASDAQ:BSTG': [datetime.date(2017, 2, 16)], 'NASDAQ:PZRX': [datetime.date(2017, 2, 2)], 'NYSEMKT:MCZ': [datetime.date(2017, 2, 3), datetime.date(2017, 2, 7), datetime.date(2017, 2, 16), datetime.date(2017, 2, 24), datetime.date(2017, 2, 27)], 'NASDAQ:MYOS': [datetime.date(2017, 1, 26), datetime.date(2017, 2, 3), datetime.date(2017, 2, 6), datetime.date(2017, 2, 7), datetime.date(2017, 2, 13), datetime.date(2017, 2, 23), datetime.date(2017, 3, 1), datetime.date(2017, 3, 2)], 'NASDAQ:ORPN': [datetime.date(2017, 1, 26)], 'NASDAQ:BLPH': [datetime.date(2017, 2, 14), datetime.date(2017, 2, 21), datetime.date(2017, 2, 22), datetime.date(2017, 2, 28)], 'NYSEMKT:NSPR': [datetime.date(2017, 2, 14)], 'NASDAQ:JAKK': [datetime.date(2017, 2, 10)], 'NASDAQ:USAK': [datetime.date(2017, 2, 8)], 'NASDAQ:GEVO': [datetime.date(2017, 2, 23)], 'NYSEMKT:NAVB': [datetime.date(2017, 2, 13), datetime.date(2017, 2, 23), datetime.date(2017, 3, 3)], 'NASDAQ:AKER': [datetime.date(2017, 2, 16)], 'NASDAQ:NVFY': [datetime.date(2017, 1, 19), datetime.date(2017, 2, 21), datetime.date(2017, 2, 22), datetime.date(2017, 2, 23), datetime.date(2017, 2, 28), datetime.date(2017, 3, 1)], 'NASDAQ:MBRX': [datetime.date(2017, 2, 1), datetime.date(2017, 2, 7), datetime.date(2017, 2, 8), datetime.date(2017, 2, 15)], 'NASDAQ:NIHD': [datetime.date(2017, 2, 1)], 'NASDAQ:ACUR': [datetime.date(2017, 2, 7), datetime.date(2017, 2, 8), datetime.date(2017, 2, 9), datetime.date(2017, 2, 10)], 'NASDAQ:EIGI': [datetime.date(2017, 2, 16)], 'NASDAQ:SINO': [datetime.date(2017, 2, 13)], 'NASDAQ:CBMX': [datetime.date(2017, 2, 2)], 'NASDAQ:VRML': [datetime.date(2017, 2, 14), datetime.date(2017, 2, 15), datetime.date(2017, 2, 21)], 'NASDAQ:RGSE': [datetime.date(2017, 1, 19), datetime.date(2017, 1, 27), datetime.date(2017, 1, 30), datetime.date(2017, 2, 1), datetime.date(2017, 2, 3), datetime.date(2017, 2, 6), datetime.date(2017, 2, 17), datetime.date(2017, 2, 23)], 'NASDAQ:CENX': [datetime.date(2017, 2, 24)], 'NASDAQ:RELV': [datetime.date(2017, 2, 14)], 'NASDAQ:SYRX': [datetime.date(2017, 1, 19), datetime.date(2017, 1, 30)], 'NASDAQ:INFI': [datetime.date(2017, 3, 2)], 'NASDAQ:PTX': [datetime.date(2017, 1, 17), datetime.date(2017, 2, 3)], 'NYSE:SGY': [datetime.date(2017, 1, 20), datetime.date(2017, 1, 23), datetime.date(2017, 2, 24)], 'NYSEMKT:TEUM': [datetime.date(2017, 2, 23), datetime.date(2017, 2, 27), datetime.date(2017, 3, 3)], 'NASDAQ:CETX': [datetime.date(2017, 3, 1)], 'NYSE:CRR': [datetime.date(2017, 1, 26)], 'NASDAQ:UNXL': [datetime.date(2017, 1, 18)], 'NYSE:EMES': [datetime.date(2017, 2, 27)], 'NASDAQ:GRVY': [datetime.date(2017, 1, 18), datetime.date(2017, 1, 19), datetime.date(2017, 2, 22)], 'NASDAQ:MSDI': [datetime.date(2017, 1, 26)], 'NASDAQ:SKLN': [datetime.date(2017, 1, 17), datetime.date(2017, 3, 1)], 'NASDAQ:PPHM': [datetime.date(2017, 2, 17)], 'NASDAQ:RCON': [datetime.date(2017, 2, 6)], 'NYSEMKT:IMUC': [datetime.date(2017, 2, 14)], 'NASDAQ:AXTI': [datetime.date(2017, 2, 23)], 'NASDAQ:ANTH': [datetime.date(2017, 1, 27), datetime.date(2017, 2, 22)], 'NASDAQ:AHPI': [datetime.date(2017, 3, 3)], 'NASDAQ:REXX': [datetime.date(2017, 1, 17), datetime.date(2017, 1, 18), datetime.date(2017, 2, 8)], 'NASDAQ:TVIA': [datetime.date(2017, 1, 19), datetime.date(2017, 2, 9), datetime.date(2017, 2, 15)], 'NASDAQ:RLOG': [datetime.date(2017, 2, 14)], 'NASDAQ:TRCH': [datetime.date(2017, 2, 27)], 'NYSE:GRAM': [datetime.date(2017, 2, 28)], 'NYSEMKT:TPLM': [datetime.date(2017, 1, 17), datetime.date(2017, 1, 20), datetime.date(2017, 1, 31)], 'NYSEMKT:CANF': [datetime.date(2017, 1, 18)], 'NASDAQ:TBIO': [datetime.date(2017, 1, 20), datetime.date(2017, 1, 26), datetime.date(2017, 1, 31), datetime.date(2017, 2, 3), datetime.date(2017, 2, 9)], 'NASDAQ:EVOK': [datetime.date(2017, 1, 30)], 'NASDAQ:SBOT': [datetime.date(2017, 3, 3)], 'NASDAQ:ACST': [datetime.date(2017, 1, 23)], 'NYSEMKT:ENRJ': [datetime.date(2017, 1, 20), datetime.date(2017, 1, 23), datetime.date(2017, 1, 26), datetime.date(2017, 2, 21)], 'NASDAQ:SBBP': [datetime.date(2017, 3, 2)], 'NASDAQ:SGNL': [datetime.date(2017, 2, 7), datetime.date(2017, 2, 13)], 'NYSEMKT:DSS': [datetime.date(2017, 2, 3), datetime.date(2017, 2, 7)], 'NASDAQ:EVAR': [datetime.date(2017, 1, 20)], 'NASDAQ:YECO': [datetime.date(2017, 1, 17)], 'NASDAQ:LNTH': [datetime.date(2017, 2, 22)], 'NYSE:BCEI': [datetime.date(2017, 1, 17)], 'NASDAQ:DVAX': [datetime.date(2017, 3, 1)], 'NASDAQ:PRZM': [datetime.date(2017, 2, 10)], 'NASDAQ:INNL': [datetime.date(2017, 1, 18), datetime.date(2017, 3, 1)], 'NASDAQ:JAGX': [datetime.date(2017, 1, 31), datetime.date(2017, 2, 1), datetime.date(2017, 2, 17), datetime.date(2017, 2, 21), datetime.date(2017, 2, 24), datetime.date(2017, 2, 27)], 'NASDAQ:LPSN': [datetime.date(2017, 2, 9)], 'NASDAQ:ARGS': [datetime.date(2017, 3, 2)], 'NASDAQ:CHCI': [datetime.date(2017, 1, 18)], 'NASDAQ:INVT': [datetime.date(2017, 1, 27), datetime.date(2017, 1, 30), datetime.date(2017, 1, 31), datetime.date(2017, 2, 15)], 'NYSEMKT:HEB': [datetime.date(2017, 2, 13)], 'NASDAQ:WRLD': [datetime.date(2017, 1, 26)], 'NYSE:SXE': [datetime.date(2017, 2, 16)], 'NASDAQ:SCON': [datetime.date(2017, 2, 16)], 'NASDAQ:PME': [datetime.date(2017, 1, 17)], 'NASDAQ:BONT': [datetime.date(2017, 2, 9)], 'NASDAQ:BASI': [datetime.date(2017, 1, 19), datetime.date(2017, 2, 14), datetime.date(2017, 2, 22), datetime.date(2017, 2, 27), datetime.date(2017, 2, 28)], 'NASDAQ:AEZS': [datetime.date(2017, 1, 17), datetime.date(2017, 1, 31)], 'NASDAQ:OCRX': [datetime.date(2017, 2, 7), datetime.date(2017, 2, 8), datetime.date(2017, 2, 9)], 'NASDAQ:FNCX': [datetime.date(2017, 1, 25), datetime.date(2017, 2, 16), datetime.date(2017, 2, 23)], 'NYSE:FAC': [datetime.date(2017, 1, 27)], 'NASDAQ:MEMP': [datetime.date(2017, 1, 17), datetime.date(2017, 1, 25), datetime.date(2017, 1, 26), datetime.date(2017, 1, 27), datetime.date(2017, 2, 15), datetime.date(2017, 2, 16)], 'NASDAQ:WKHS': [datetime.date(2017, 1, 26)], 'NASDAQ:EYES': [datetime.date(2017, 2, 7), datetime.date(2017, 2, 10)], 'NASDAQ:SEV': [datetime.date(2017, 2, 3)], 'NASDAQ:ALQA': [datetime.date(2017, 3, 2)], 'NYSEMKT:NAK': [datetime.date(2017, 2, 21), datetime.date(2017, 2, 22), datetime.date(2017, 2, 23)], 'NASDAQ:VIVO': [datetime.date(2017, 1, 25)], 'NASDAQ:VNRAP': [datetime.date(2017, 1, 25), datetime.date(2017, 2, 2), datetime.date(2017, 2, 3)], 'NASDAQ:STML': [datetime.date(2017, 2, 2)], 'NASDAQ:PRIM': [datetime.date(2017, 2, 28)], 'NYSE:OB': [datetime.date(2017, 1, 18)], 'NASDAQ:BCLI': [datetime.date(2017, 2, 1)], 'NASDAQ:KBSF': [datetime.date(2017, 2, 9), datetime.date(2017, 2, 10), datetime.date(2017, 2, 15), datetime.date(2017, 2, 16), datetime.date(2017, 2, 22), datetime.date(2017, 2, 23)], 'NASDAQ:BNTC': [datetime.date(2017, 1, 17), datetime.date(2017, 2, 2), datetime.date(2017, 2, 3), datetime.date(2017, 2, 7), datetime.date(2017, 2, 15), datetime.date(2017, 2, 16)], 'NYSE:CHKR': [datetime.date(2017, 1, 25)], 'NYSEMKT:APHB': [datetime.date(2017, 2, 8)], 'NYSEMKT:TPHS': [datetime.date(2017, 2, 28)], 'NASDAQ:STAF': [datetime.date(2017, 2, 17)], 'NYSE:BNED': [datetime.date(2017, 2, 28)], 'NASDAQ:SNMX': [datetime.date(2017, 2, 27)], 'NYSEMKT:SSN': [datetime.date(2017, 2, 21)], 'NASDAQ:CRDS': [datetime.date(2017, 1, 19), datetime.date(2017, 2, 13)], 'NASDAQ:NVAX': [datetime.date(2017, 2, 15)], 'NASDAQ:FATE': [datetime.date(2017, 2, 17)], 'NASDAQ:MBVX': [datetime.date(2017, 2, 23)], 'NASDAQ:BIOL': [datetime.date(2017, 1, 17)], 'NASDAQ:TOPS': [datetime.date(2017, 2, 16), datetime.date(2017, 2, 24)], 'NASDAQ:URRE': [datetime.date(2017, 1, 19)], 'NASDAQ:APOP': [datetime.date(2017, 2, 2), datetime.date(2017, 2, 8), datetime.date(2017, 2, 14), datetime.date(2017, 2, 21)]}

#modelFunc = screener.ModelintVdPSharpFall
#modelFunc = screener.ModelintVdPStopThenIncrease
modelFunc = screener.ModelPriceOnly_SharpRise
params = {'df':screener.paramTypes.intradaylive}
screener.TestModels(modelFunc, params, symbols_dates=noticable_dic, day_start=None, \
            day_end=None, price_change_perc=[3, 10])

In [ ]:
%pylab inline
from Finance.Finance import *
import datetime
import numpy as np

screener = StockScreener()

#NASDAQ:APOP NYSEMKT:XTNT
symb = r'NASDAQ:PULM'
df = screener.plotSymbolIntraday(symb, duration=[10, 'd'], extplt=False, extpltData=[60, 4])
print(df[-10:])

figsize(10, 4)

day_open = datetime.datetime(2017, 1, 26, 9, 30)
day_close = datetime.datetime(2017, 1, 26, 16, 0)

df = df[np.logical_and(df.index>=day_open, df.index<=day_close)]
plot((df['CLOSE']-df['CLOSE'].min())/(df['CLOSE'].max()-df['CLOSE'].min()), 'b', lw=1.0)
#plot(df['VOLUME']/df['VOLUME'].max(), 'r', lw=0.4)
bar(df.index.values, (df['VOLUME']/df['VOLUME'].max()).values, width=0.0002, color='b', linewidth=0)

### int(dpdv)
P = df['CLOSE'].values
V = df['VOLUME'].values
T = df.index.values

dP = P[1:]-P[:-1]
dV = V[1:]-V[:-1]
dT = T[1:]-T[:-1]

dPV = P[1:]*V[1:]-P[:-1]*V[:-1]
VdP = V[1:]*dP
intVdP = np.zeros(len(VdP))
intVdP[0] = VdP[0]
for i in range(1, len(VdP)):
    intVdP[i] = intVdP[i-1]+VdP[i]

    
#plot(T[1:], dPV/np.abs(dPV).max(), 'k', lw=1.0)
#plot(T[1:], VdP/np.abs(VdP).max(), 'g:')
plot(T[1:], intVdP/np.abs(intVdP).max(), 'r')

##mark 5 minutes flat region with less than 2% change
##find increasing volume with more than 3% change in 2 minutes
## calculate the positive and negative momentum

intVdP_rel = intVdP/np.abs(intVdP).max()
dintdVP_rel = intVdP_rel[1:]-intVdP_rel[:-1]

plot(T[2:], dintdVP_rel/np.abs(dintdVP_rel).max(), 'k', lw=1.0)


import pandas as pd
from pandas.tseries.offsets import BDay

date_prev = day_open.date() - BDay(1)
print('Previous day: ', date_prev)
intraday_prev = screener.GetIntraDayMinuteTradingInfo(symb, date_prev, mins_max=20)
V_prev_mean, V_prev_max = None, None
if intraday_prev!=None:
    V_prev_mean, V_prev_max = intraday_prev['volume-avg'], intraday_prev['volume-max']

print('V_mean_prev: \n', V_prev_mean/V.max())
print('V_max_prev : \n', V_prev_max/V.max())
    
n_flat = 5
for i in range(n_flat, len(VdP)):
    if np.abs(dintdVP_rel[i-n_flat:i+1]).max()<0.01:
        axvline(T[i], color='y')

for i in range(1, len(dP)):
    if V[i]/V[:i].mean()>1.0 and dV[i]/V[:i].mean()>1.0 and dP[i]/P[:i].mean()>0.02:
        axvline(T[i], color='g')
        
show()

## based on the average last day and today's volume and price find the appropriate thresholds
## use t, t-1 ... t-n for preditions

plot(T, (P-P.min())/(P.max()-P.min()), 'b', lw=1.0)
#bar(T, V/V.max(), width=0.0002, color='b', linewidth=0.00001)
plot(T, V/V.max(), color='g', linewidth=0.4)
plot(T[1:], intVdP/np.abs(intVdP).max(), 'r')


In [1]:
from Finance.Finance import StockScreener

screener = StockScreener()

#screener.UpdateIntraday30minStockData_All()

print('='*80)
screener.UpdateIntradayStockData_All()


##---- shutdown
shutdown = False
if shutdown:
    import subprocess
    import time
    
    print('shutting down in 10 minutes.')
    time.sleep(10*60)
    print('shutting down...')
    p = subprocess.Popen(['shutdown'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, err = p.communicate()
    print(out)


NASDAQ:PIH:0/3175 NASDAQ:FLWS:1/3175 NASDAQ:FCCY:2/3175 NASDAQ:SRCE:3/3175 NASDAQ:VNET:4/3175 NASDAQ:TWOU:5/3175 NASDAQ:JOBS:6/3175 NASDAQ:CAFD:7/3175 NASDAQ:EGHT:8/3175 NASDAQ:AVHI:9/3175 NASDAQ:SHLM:10/3175 NASDAQ:AAON:11/3175 NASDAQ:ABAX:12/3175 NASDAQ:ABEO:13/3175 NASDAQ:ABEOW:14/3175 NASDAQ:ABIL:15/3175 NASDAQ:ABMD:16/3175 NASDAQ:AXAS:17/3175 NASDAQ:ACIU:18/3175 NASDAQ:ACIA:19/3175 NASDAQ:ACTG:20/3175 NASDAQ:ACHC:21/3175 NASDAQ:ACAD:22/3175 NASDAQ:ACST:23/3175 NASDAQ:AXDX:24/3175 NASDAQ:XLRN:25/3175 NASDAQ:ANCX:26/3175 NASDAQ:ARAY:27/3175 NASDAQ:ACRX:28/3175 NASDAQ:ACET:29/3175 NASDAQ:AKAO:30/3175 NASDAQ:ACHN:31/3175 NASDAQ:ACIW:32/3175 NASDAQ:ACRS:33/3175 NASDAQ:ACNB:34/3175 NASDAQ:ACOR:35/3175 NASDAQ:ACTS:36/3175 NASDAQ:ATVI:37/3175 NASDAQ:ACTA:38/3175 NASDAQ:ACUR:39/3175 NASDAQ:ACXM:40/3175 NASDAQ:ADMS:41/3175 NASDAQ:ADMP:42/3175 NASDAQ:ADAP:43/3175 NASDAQ:ADUS:44/3175 NASDAQ:AEY:45/3175 NASDAQ:IOTS:46/3175 NASDAQ:ADMA:47/3175 NASDAQ:ADBE:48/3175 NASDAQ:ADTN:49/3175 NASDAQ:ADRO

In [ ]:
from Finance.Finance import StockScreener

screener = StockScreener()

df = screener.GetLastDayTradingInfo('NASDAQ:AMD')

In [ ]:
import datetime
print(df)


### Calculator

In [7]:
## USD
allocated_fund = 1300
print('allocated fund: {} USD'.format(allocated_fund))
stock_price = 0.5001
print('stck price: {} USD'.format(stock_price))

print('-'*50)
for n in range(10):
    stock_price_n_low = stock_price*(100-n)/100
    n_stocks_n_low = int(allocated_fund/stock_price_n_low)
    print('stck price {}% down: {:8.4} USD    N: {}'.format(n, stock_price_n_low, n_stocks_n_low))
print('-'*50)
for n in range(10):
    stock_price_n_up = stock_price*(100+n)/100
    n_stocks_n_up = int(allocated_fund/stock_price_n_up)
    print('stck price {}% up  : {:8.4} USD    N: {}'.format(n, stock_price_n_up, n_stocks_n_up))



allocated fund: 1300 USD
stck price: 0.5001 USD
--------------------------------------------------
stck price 0% down:   0.5001 USD    N: 2599
stck price 1% down:   0.4951 USD    N: 2625
stck price 2% down:   0.4901 USD    N: 2652
stck price 3% down:   0.4851 USD    N: 2679
stck price 4% down:   0.4801 USD    N: 2707
stck price 5% down:   0.4751 USD    N: 2736
stck price 6% down:   0.4701 USD    N: 2765
stck price 7% down:   0.4651 USD    N: 2795
stck price 8% down:   0.4601 USD    N: 2825
stck price 9% down:   0.4551 USD    N: 2856
--------------------------------------------------
stck price 0% up  :   0.5001 USD    N: 2599
stck price 1% up  :   0.5051 USD    N: 2573
stck price 2% up  :   0.5101 USD    N: 2548
stck price 3% up  :   0.5151 USD    N: 2523
stck price 4% up  :   0.5201 USD    N: 2499
stck price 5% up  :   0.5251 USD    N: 2475
stck price 6% up  :   0.5301 USD    N: 2452
stck price 7% up  :   0.5351 USD    N: 2429
stck price 8% up  :   0.5401 USD    N: 2406
stck price 9% 

In [ ]:
from Finance.Finance import StockScreener

import numpy as np
import os
cwd = os.getcwd()
name = 'rbc.npz'
file_encrypted = os.path.join(cwd, 'attachments', name+'.dec')
PASS = "secretpassbabe"

screener = StockScreener()
screener.FillInRBCOrder(file_encrypted, PASS, account='TSFS:USD', symbol='UNXL', market='US', \
            order='sell', num_shares=2717, price=0.345, AllorNone=False)

In [5]:
%pylab wx

from Finance.Finance import *
import datetime
import numpy as np
import pandas as pd

screener = StockScreener()


noticable_dic = {'NASDAQ:ZSAN': [datetime.date(2017, 1, 31), datetime.date(2017, 2, 13), datetime.date(2017, 2, 14), datetime.date(2017, 2, 15), datetime.date(2017, 2, 16), datetime.date(2017, 2, 24)]}


figsize(10,4)

for symb in noticable_dic:
    for day in noticable_dic[symb]:
        df = screener.GetIntradayMinutData(symb, day)
        if isinstance(df, pd.DataFrame):
            #print(df.head())
            P = df['CLOSE'].values
            V = df['VOLUME'].values
            T = df.index.values
            plot(T, P)
show()


Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.5/dist-packages/ipykernel/kernelbase.py:390: FutureWarning: passing multiple values to Expr is deprecated, pass the where as a single string
  user_expressions, allow_stdin)
